### Imports 

In [3]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import gurobipy as gb
import numpy as np

In [5]:
class PDSVRPModel:

    def __init__(self, instance):
        self.C_T = instance.C_T #truck transportation cost
        self.C_D = instance.C_D #drone transportation cost
        self.D = instance.D #number of drones
        self.N = instance.N #number of nodes
        self.h = instance.h #number of trucks
        self.distances= instance.distances #matrix of euclidean distances
        self.manhattan_distances = instance.manhattan_distances #matrix of manhattan distances
        self.t_t = instance.t_t #matrix of trucks travel times
        self.t_d = instance.t_d #vector of drones travel times
        self.Q_t = instance.Q_t #truck capacity
        self.Q_d = instance.Q_d #drone capacity
        self.T_t = instance.T_t #max time truck
        self.T_d = instance.T_d #max time drones
        self.w = instance.w #weights vector
        self.d_end = instance.d_end #drone endurance
        
        self.model = gb.Model("OptimizationModel")
        self.x = None
        self.y = None
        self.z = None
        self.u = None

        
    def build_model(self):
        # Define the decision variables
        self.x = self.model.addVars([(i,j) for i in range(self.N) for j in range(self.N) if i!=j], vtype=gb.GRB.BINARY, name="x")#ho messo +1 perchè lo 0 è il depot e poi ci sono N clienti
        self.y = self.model.addVars([(i,k) for i in range(1, self.N) for k in range(self.D)], vtype=gb.GRB.BINARY, name="y")#ok
        self.z = self.model.addVars([(i,j) for i in range(self.N) for j in range(self.N) if i!=j], lb=0, ub=self.T_t, vtype=gb.GRB.CONTINUOUS, name="z")#ok (lb e ub sono già specificati nei constraits) 
        self.u = self.model.addVars([(i) for i in range(1, self.N)], lb=0, ub=self.Q_t, vtype=gb.GRB.CONTINUOUS, name="u")#ok

        # Define the objective function
        self.model.setObjective(
            gb.quicksum(self.C_T * self.manhattan_distances[i,j] * self.x[i, j] for i in range (self.N) for j in range (self.N) if i != j) +
            gb.quicksum(self.C_D * 2 * self.distances[0,k] * self.y[k, l] for k in range (1, self.N) for l in range(self.D)),
            gb.GRB.MINIMIZE
        ) #ok

        N_t = [i for i in range(1,self.N) if (self.w[i] > self.Q_d or ((self.t_d[i] * 2) > self.d_end))] #indexes of clients that must be served by trucks
        N_f= [i for i in range(1,self.N) if i not in N_t] #indexes of clients that can ber served either by e truck or a drone

        # Add constraints
        self.model.addConstr(gb.quicksum(self.x[0, i] for i in range (1, self.N)) <= self.h, "2) Constraint on number of trucks") #ok

        for j in range(self.N):
            self.model.addConstr(gb.quicksum(self.x[i, j] for i in range(self.N) if i != j) == gb.quicksum(self.x[j, i] for i in range(self.N) if i != j), "3) Flow constraint") #ok

        for j in N_f:
            self.model.addConstr(gb.quicksum(self.x[i, j] for i in range(self.N) if i != j) + gb.quicksum(self.y[j, k] for k in range(self.D)) == 1, "4) Every customer in N_f must be served") #ok

        for j in N_t:
            self.model.addConstr(gb.quicksum(self.x[i, j] for i in range(self.N) if i != j) == 1, "5) Truck customers must be visited by only trucks") #ok

        for i in range (1, self.N):
            for j in range (1, self.N):
                if j != i:
                    self.model.addConstr(self.u[i] - self.u[j] + self.Q_t * self.x[i, j] <= self.Q_t - self.w[j], "6) Miller-Tucker_Zemlin constraint") #ok

        for k in range(self.D):
            self.model.addConstr(gb.quicksum(self.y[j, k] * self.t_d[j] for j in N_f) <= self.T_d, "7) Time constraint for drones") #ok

        for i in range(1, self.N):
            self.model.addConstr(gb.quicksum(self.z[l, i] for l in range(self.N) if l != i) + gb.quicksum(self.t_t[i, j] * self.x[i, j] for j in range(self.N) if j != i) == gb.quicksum(self.z[i, j] for j in range(self.N) if j != i),"8) Inductive step for induction method of cumulative time additions")

        for i in range(1, self.N):
            self.model.addConstr(self.z[0, i] == self.t_t[0, i] * self.x[0, i], "9) Basic step for induction method of cumulative time additions")

        for i in range(1, self.N):
            self.model.addConstr(self.z[i, 0] <= self.T_t * self.x[i, 0], "10) Truck time limit constraint")

        # Optimization parameters
        self.model.setParam('Threads', 8) #Set number of threads
        #self.model.setParam('MIPGap', 0.01)  # Set tolerance
        #self.model.setParam('MIPFocus', 3)  # Set focus
        self.model.setParam('Presolve', 2)  # Presolve level increase
        self.model.setParam('Cuts', 3)  # Aggressive cuts
        self.model.setParam('TimeLimit', 3600) # Time limit of an hour

    def solve(self):
        self.model.optimize()

    def print_results(self):
        if self.model.status == gb.GRB.OPTIMAL:
            for v in self.model.getVars():
                if v.x != 0:
                    print(f'{v.varName}: {v.x}')
            print("Non printed variables equal 0")
            print(f'Obj: {self.model.objVal}')
            print(f'Time taken: {self.model.Runtime} seconds')
        else:
            print("No optimal solution found.")
            print(f'Time taken: {self.model.Runtime} seconds')
            
    def get_resolution_time(self):
        return self.model.Runtime

In [7]:
import PDSVRP_instance

def test_solver(instance_file_path):
    instance = PDSVRP_instance.PDSVRPInstance(instance_file_path)
    model = PDSVRPModel(instance) 
    model.build_model()
    model.solve()
    model.print_results()

In [8]:
test_solver('instances/30-c-0-c.txt')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-21
Set parameter Threads to value 8
Set parameter Presolve to value 2
Set parameter Cuts to value 3
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1026 rows, 2010 columns and 8412 nonzeros
Model fingerprint: 0x1d43c55e
Variable types: 960 continuous, 1050 integer (1050 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+03]
  Objective range  [6e-02, 3e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 161.5864695
Presolve removed 30 rows and 489 columns
Presolve time: 0.04s
Presolved: 996 rows, 1521 columns, 10384 nonzeros
Variable types: 495 continuous, 1026 intege

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\giova\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\iostream.py", line 655, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


H48844 37531                      81.4616768   39.07424  52.0%   6.8   14s
 51545 40260   41.86564   50   20   81.46168   39.16718  51.9%   6.9   15s
